In [1]:
# Imports

import  pandas as pd
import os
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score,recall_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# import xgboost as xgbm
# import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve
import seaborn as sns
print("Imported All Imports")

Imported All Imports


In [2]:
train = pd.read_excel('train.xlsx',sheetno=0)
test = pd.read_excel('test.xlsx',sheetno=0)
sample = pd.read_excel('sample.xlsx',sheetno=0)

In [3]:
train.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [4]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


### Detect Missing Values and Handling Missing Values

In [6]:
def missing_values_table(df):
	mis_val = df.isnull().sum()
	mis_val_percent = 100 * df.isnull().sum() / len(df)
	mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
	mis_val_table_ren_columns = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})
	mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values('% of Total Values', ascending=False).round(2)
	print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n There are " + str(mis_val_table_ren_columns.shape[0]) + " columns that have missing values.")
	return mis_val_table_ren_columns

In [7]:
missing_values_table(train)

Your selected dataframe has 11 columns.
 There are 2 columns that have missing values.


,Missing Values,% of Total Values
Route,1,0.01
Total_Stops,1,0.01


In [8]:
missing_values_table(test)

Your selected dataframe has 10 columns.
 There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [9]:
# As the data contains only one missing row, so deleting it
def handleMissingValues(data):
    data.dropna(axis=0, how='any', subset=['Route', 'Total_Stops'],inplace=True)
    return data

In [10]:
train = handleMissingValues(train)

In [11]:
# Label Encoding of Variable
def labelEncoding(data,columns):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for i in columns:
        data[i] = le.fit_transform(data[i])
    return data

In [12]:
train = labelEncoding(train,['Airline','Source','Destination','Total_Stops','Additional_Info'])

In [13]:
test = labelEncoding(test,['Airline','Source','Destination','Total_Stops','Additional_Info'])

### Feature Engineering started

In [14]:
def cal_duration_hours(df):
    if len(df['Duration'].split()) !=2:
        if 'h' in df['Duration']:
            return int(df['Duration'].split()[0][:-1])
        else:
            return 0
    else:
        if 'h' in df['Duration']:
            return int(df['Duration'].split()[0][:-1])

In [15]:
def cal_duration_mins(df):
    if len(df['Duration'].split()) !=2:
        if 'm' in df['Duration']:
            return int(df['Duration'].split()[0][:-1])
        else:
            return 0
    else:
        if 'm' in df['Duration']:
            return int(df['Duration'].split()[1][:-1])

In [16]:
def addNewFeatures(data):
    data['journey_month'] = pd.to_datetime(data['Date_of_Journey'],format='%d/%m/%Y').dt.month
    data['journey_day'] = pd.to_datetime(data['Date_of_Journey'],format='%d/%m/%Y').dt.day
    data['duration_hours'] =  data.apply(cal_duration_hours,axis=1)
    data['duration_mins'] =  data.apply(cal_duration_mins,axis=1)
    data['Arrival_Time_Hour'] = pd.to_datetime(data['Arrival_Time']).dt.hour 
    data['Arrival_Time_mins'] = pd.to_datetime(data['Arrival_Time']).dt.minute
    data['Dep_Time_Hour'] = pd.to_datetime(data['Dep_Time']).dt.hour 
    data['Dep_Time_mins'] = pd.to_datetime(data['Dep_Time']).dt.minute
    data.drop(['Date_of_Journey','Arrival_Time','Duration','Dep_Time'],axis=1,inplace=True)
    return data


In [17]:
train = addNewFeatures(train)

In [18]:
test = addNewFeatures(test)

In [19]:
train.shape

(10682, 15)

In [20]:
test.shape

(2671, 14)

In [21]:
X = train.drop(['Price'],axis=1)
Y = np.array(train['Price'].tolist())

In [22]:
X['is_test'] = 0
test['is_test'] = 1
total = pd.concat([X,test],axis=0,sort=False)

In [23]:
total['Route'].unique()

array(['BLR → DEL', 'CCU → IXR → BBI → BLR', 'DEL → LKO → BOM → COK',
       'CCU → NAG → BLR', 'BLR → NAG → DEL', 'CCU → BLR',
       'BLR → BOM → DEL', 'DEL → BOM → COK', 'DEL → BLR → COK',
       'MAA → CCU', 'CCU → BOM → BLR', 'DEL → AMD → BOM → COK',
       'DEL → PNQ → COK', 'DEL → CCU → BOM → COK', 'BLR → COK → DEL',
       'DEL → IDR → BOM → COK', 'DEL → LKO → COK',
       'CCU → GAU → DEL → BLR', 'DEL → NAG → BOM → COK',
       'CCU → MAA → BLR', 'DEL → HYD → COK', 'CCU → HYD → BLR',
       'DEL → COK', 'CCU → DEL → BLR', 'BLR → BOM → AMD → DEL',
       'BOM → DEL → HYD', 'DEL → MAA → COK', 'BOM → HYD',
       'DEL → BHO → BOM → COK', 'DEL → JAI → BOM → COK',
       'DEL → ATQ → BOM → COK', 'DEL → JDH → BOM → COK',
       'CCU → BBI → BOM → BLR', 'BLR → MAA → DEL',
       'DEL → GOI → BOM → COK', 'DEL → BDQ → BOM → COK',
       'CCU → JAI → BOM → BLR', 'CCU → BBI → BLR', 'BLR → HYD → DEL',
       'DEL → TRV → COK', 'CCU → IXR → DEL → BLR',
       'DEL → IXU → BOM → COK', 'CCU 

In [24]:
total.shape

(13353, 15)

In [25]:
df = total['Route'].str.split('→', expand=True)
df[0]=df[0].str.strip()
df[1]=df[1].str.strip()
df[2]=df[2].str.strip()
df[3]=df[3].str.strip()
df[4]=df[4].str.strip()
df[5]=df[5].str.strip()

In [26]:
df.shape

(13353, 6)

In [27]:
df = pd.get_dummies(df, prefix='', prefix_sep='')

In [28]:
df.head()

,BLR,BOM,CCU,DEL,MAA,AMD,ATQ,BBI,BDQ,BHO,...,GWL,HYD,NAG,TRV,BLR,COK,DEL,HYD,VGA,DEL
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df = df.groupby(df.columns, axis=1).sum()

In [30]:
df.columns

Index(['AMD', 'ATQ', 'BBI', 'BDQ', 'BHO', 'BLR', 'BOM', 'CCU', 'COK', 'DED',
       'DEL', 'GAU', 'GOI', 'GWL', 'HBX', 'HYD', 'IDR', 'IMF', 'ISK', 'IXA',
       'IXB', 'IXC', 'IXR', 'IXU', 'IXZ', 'JAI', 'JDH', 'JLR', 'KNU', 'LKO',
       'MAA', 'NAG', 'NDC', 'PAT', 'PNQ', 'RPR', 'STV', 'TIR', 'TRV', 'UDR',
       'VGA', 'VNS', 'VTZ'],
      dtype='object')

In [31]:
df.shape

(13353, 43)

In [32]:
total = pd.concat([total, df], axis=1)

In [33]:
total.columns

Index(['Airline', 'Source', 'Destination', 'Route', 'Total_Stops',
       'Additional_Info', 'journey_month', 'journey_day', 'duration_hours',
       'duration_mins', 'Arrival_Time_Hour', 'Arrival_Time_mins',
       'Dep_Time_Hour', 'Dep_Time_mins', 'is_test', 'AMD', 'ATQ', 'BBI', 'BDQ',
       'BHO', 'BLR', 'BOM', 'CCU', 'COK', 'DED', 'DEL', 'GAU', 'GOI', 'GWL',
       'HBX', 'HYD', 'IDR', 'IMF', 'ISK', 'IXA', 'IXB', 'IXC', 'IXR', 'IXU',
       'IXZ', 'JAI', 'JDH', 'JLR', 'KNU', 'LKO', 'MAA', 'NAG', 'NDC', 'PAT',
       'PNQ', 'RPR', 'STV', 'TIR', 'TRV', 'UDR', 'VGA', 'VNS', 'VTZ'],
      dtype='object')

In [34]:
total.drop(['Route'],inplace=True,axis=1)

In [35]:
X = total[total['is_test'] == 0]
test = total[total['is_test'] == 1]

In [36]:
X.shape

(10682, 57)

In [37]:
test.shape

(2671, 57)

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [39]:
X_train.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,journey_month,journey_day,duration_hours,duration_mins,Arrival_Time_Hour,...,PAT,PNQ,RPR,STV,TIR,TRV,UDR,VGA,VNS,VTZ
8454,4,2,1,4,5,5,27,3,15,14,...,0,0,0,0,0,0,0,0,0,0
8423,4,2,1,0,5,5,9,11,0,19,...,0,0,0,0,0,0,0,0,0,0
10179,6,2,1,0,8,6,3,9,5,16,...,0,0,0,0,0,0,0,0,0,0
3880,4,3,0,0,5,6,9,27,35,23,...,0,0,0,0,0,0,0,0,0,0
6176,4,2,1,0,5,6,27,7,30,4,...,0,0,0,0,0,0,0,0,0,0


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train, Y_train)
y_test_pred_rf = rf.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,y_test_pred_rf)))

1468.9207349499065


In [42]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, Y_train)
y_test_pred_lr = lr.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,y_test_pred_lr)))

2880.9842849719107
